In [ ]:
# Load country-specific data
df = pd.read_csv('../../data/processed/household_survey/Nigeria.csv')

# Load conflict data
conflict = pd.read_excel('../../data/processed/conflict/Nigeria.xlsx')
df = pd.merge(df, conflict, on='region')

# The "starting point" is the expenditure
# From there we can calculate everything else

# First, on the country level without taking into account conflict
# Clip exp to q1 and q3
q1 = df['exp'].quantile(0.01)
q3 = df['exp'].quantile(0.99)
df['exp'] = df[(df['exp'] > q1) & (df['exp'] < q3)]['exp']

weighted_mean = (df['exp'] * df['wgt']).sum() / df['wgt'].sum()
weighted_min = ((df['exp'] * df['wgt']) / df['wgt']).min()
weighted_max = ((df['exp'] * df['wgt']) / df['wgt']).max()

print("Weighted Mean:", round(weighted_mean))
print("Weighted Minimum:", round(weighted_min))
print("Weighted Maximum:", round(weighted_max))

In [ ]:
mean_vulnerability = df.groupby('conflict_intensity').mean(numeric_only=True)[['v_init']]
mean_vulnerability = mean_vulnerability.rename(columns={'v_init': 'mean_vulnerability'})
mean_vulnerability.index = pd.Categorical(mean_vulnerability.index, categories=['Very low', 'Low', 'Medium', 'High', 'Very high'], ordered=True)
mean_vulnerability.sort_index(inplace=True)
mean_vulnerability = mean_vulnerability.round(2)
mean_vulnerability.to_dict()

exp_by_conflict = df.groupby('conflict_intensity').describe()['exp']
exp_by_conflict.index = pd.Categorical(exp_by_conflict.index, categories=['Very low', 'Low', 'Medium', 'High', 'Very high'], ordered=True)
exp_by_conflict.sort_index(inplace=True)
exp_by_conflict = exp_by_conflict.round(2)
exp_by_conflict[['mean', 'std', 'min', 'max']].T.to_dict()